In [1]:
!pip install google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 265.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 570.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 590.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 877.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 723.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 571.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 778.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 709.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 534.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 520.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 895.9 kB/s eta 0:00:00a 0:00:01
  Attemptin

## csv to bigquery
-- 參考: https://estuary.dev/csv-to-bigquery/

In [41]:
from google.cloud import bigquery
import os

class BigQueryLoader:
    def __init__(self):
        self.dataset_name = 'pdata'
        self.table_name = None
        self.schema_setting = None
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./config/skyscannerweb-3dc95bba3ffb.json"
        self.client = bigquery.Client()
    def set_table_schema(self, table_name, schema_setting):
        self.table_name = table_name
        self.schema_setting = schema_setting
        self.table_ref = self.client.dataset(self.dataset_name).table(self.table_name)
        self.job_config = bigquery.LoadJobConfig(**self.schema_setting)
    def load_data_from_csv(self, file_path):
        if self.table_name is None or self.schema_setting is None:
            raise ValueError("Table name and schema setting must be set before loading data.")
        with open(file_path, 'rb') as source_file:
            job = self.client.load_table_from_file(source_file, self.table_ref, job_config=self.job_config)
            job.result()  # Wait for the job to complete
            print('Done')
if __name__ == '__main__':
    BigQueryLoader()

In [9]:
import os
from tools.data_importer import BigQueryLoader
from config.bigquery_config import schema_setting

def get_latest_file_path(prefix, folder):
    files = [f for f in os.listdir(folder) if f.startswith(prefix)]
    latest_file = sorted(files, key=lambda x: os.path.getmtime(os.path.join(folder, x)), reverse=True)[0]
    print(f"最新創建的{prefix}檔案是: {latest_file}")
    return os.path.join(folder, latest_file)

def load_data_to_bigquery(table_name, schema, data_path):
    loader = BigQueryLoader()
    loader.set_table_schema(table_name, schema)
    loader.load_data_from_csv(data_path)
    
if __name__ == "__main__":
    main_path = os.getcwd()  # 程式所在的目錄路徑
    data_path = 'data'
    combine_path = os.path.join(main_path, data_path)  # 檔案所在的目錄路徑

    # flight_price
    table_name_flight_price = 'flight_price'
    flight_price_prefix = "flight_price"  # 檔案名稱前綴
    flight_price_data_path = get_latest_file_path(flight_price_prefix, combine_path)
    load_data_to_bigquery(table_name_flight_price, schema_setting['flight_price'], flight_price_data_path)

    # flight_memo
    table_name_flight_memo = 'flight_memo'
    flight_memo_prefix = "flight_memo"  # 檔案名稱前綴
    flight_memo_data_path = get_latest_file_path(flight_memo_prefix, combine_path)
    load_data_to_bigquery(table_name_flight_memo, schema_setting['memo'], flight_memo_data_path)

    # flight_go_plan
    table_name_flight_go_plan = 'flight_go_plan'
    flight_go_plan_prefix = "go_plan"  # 檔案名稱前綴
    flight_go_plan_data_path = get_latest_file_path(flight_go_plan_prefix, combine_path)
    load_data_to_bigquery(table_name_flight_go_plan, schema_setting['flight_go_plan'], flight_go_plan_data_path)

最新創建的flight_price檔案是: flight_price_20230803154719.csv
Done
最新創建的flight_memo檔案是: flight_memo_20230803154719.csv
Done
最新創建的go_plan檔案是: go_plan_20230803154719.csv
Done
